In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import os
import sys
sys.path.insert(0, os.path.dirname('../'))

from config import Config
from data_utils import video_to_frames
from data_utils import metadata_loader
from data_utils import dataset_builder

from models.IMAGENET import Imagenet, Video_Feature_Extractor 
from models.IMAGENET import AVG_Video_Classifier, LSTM_Video_Classifier

In [2]:
config = Config(
    root_path='../data/20bn-something-something-v2', 
    json_path='./data/20bn-something-something-v2-jason')


# Get metadata
ml = metadata_loader.MetadataLoader(config)
metadata = ml.load_metadata()

# Build datasets
db = dataset_builder.DatasetBuilder(config)
train_dataset = db.make_frame_dataset(metadata['train'])
valid_dataset = db.make_frame_dataset(metadata['valid'])
test_dataset = db.make_frame_dataset(metadata['test'])

### Resize images to 160x160

In [3]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
    #image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [4]:
train = train_dataset.map(format_example)
validation = valid_dataset.map(format_example)
test = test_dataset.map(format_example)

### Shuffle and batch

In [5]:
BATCH_SIZE = 1
SHUFFLE_BUFFER_SIZE = 10

In [6]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    print(image_batch.shape)

(1, 160, 160, 3)


# Get pretrained model on Imagenet

In [9]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Get the base model from the pre-trained InceptionV3
base_model = Imagenet(input_shape=IMG_SHAPE, name='inception')
base_model.trainable = False

print(" Input batch shape:",image_batch.shape)
feature_batch = base_model(image_batch)
print("Output batch shape:",feature_batch.shape)
# freeze weights


 Input batch shape: (1, 160, 160, 3)
Output batch shape: (1, 3, 3, 2048)


In [12]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

# 2) Train new classifier with features from base model

###  Add the extra layers on top of pretrained base model
1) Global pooling and averaging

2) Last classifieng layer

In [13]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() ##
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(1, 2048)


In [14]:
prediction_layer = tf.keras.layers.Dense(config.n_classes) ##
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(1, 174)


In [15]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [16]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
model.summary()
print("Number of trainabe layers:", np.sum([l.trainable for l in model.layers]))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 174)               356526    
Total params: 22,159,310
Trainable params: 356,526
Non-trainable params: 21,802,784
_________________________________________________________________
Number of trainabe layers: 2


### Check starting accuracy

In [19]:
initial_epochs = 2
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 2s 94ms/step - loss: 0.8028 - accuracy: 0.7008
initial loss: 0.80
initial accuracy: 0.70


### Fit

In [20]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

Train for 1148 steps, validate for 166 steps
Epoch 1/2
1148/1148 [==============================] - 57s 50ms/step - loss: 0.0635 - accuracy: 0.9924 - val_loss: 0.0438 - val_accuracy: 0.9943
Epoch 2/2
1148/1148 [==============================] - 55s 48ms/step - loss: 0.0532 - accuracy: 0.9943 - val_loss: 0.0450 - val_accuracy: 0.9943


### Check accuracy after training

In [22]:
loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 1s 38ms/step - loss: 0.0436 - accuracy: 0.9943
initial loss: 0.04
initial accuracy: 0.99


# 3) Fine tune by training deeper layers
Here we finetune all layers after the 100th layer. 
Normally for the inception you would finetune the 1,2 or 3 last inception blocks

In [23]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("--> Number of layers in the base model:", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_from = 100

# Freeze all the layers before the `fine_tune_from` layer
for layer in base_model.layers[:fine_tune_from]:
    layer.trainable =  False
    
print("--> Number of trainabe layers:", np.sum([l.trainable for l in base_model.layers]))
print("\n")
model.summary()



--> Number of layers in the base model: 155
--> Number of trainabe layers: 55


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 174)               222894    
Total params: 618,286
Trainable params: 222,894
Non-trainable params: 395,392
_________________________________________________________________


In [25]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [26]:
fine_tune_epochs = 2
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

Train for 1148 steps, validate for 166 steps
Epoch 2/4
1148/1148 [==============================] - 80s 69ms/step - loss: 4.2694 - accuracy: 0.1910 - val_loss: 6.6932 - val_accuracy: 0.0043
Epoch 3/4
1148/1148 [==============================] - 75s 65ms/step - loss: 3.2260 - accuracy: 0.3893 - val_loss: 7.0584 - val_accuracy: 0.0043
Epoch 4/4
1148/1148 [==============================] - 76s 66ms/step - loss: 2.5013 - accuracy: 0.5623 - val_loss: 6.8984 - val_accuracy: 0.0043


In [27]:
loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 1s 40ms/step - loss: 6.3908 - accuracy: 0.0000e+00
initial loss: 6.39
initial accuracy: 0.00
